---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [5]:
import pandas as pd
import numpy as np

def blight_model():
    # Load the data files
    train = pd.read_csv('train.csv',encoding='ISO-8859-1',low_memory=False,parse_dates=['ticket_issued_date', 'hearing_date','payment_date'])
    test  = pd.read_csv('test.csv',encoding='ISO-8859-1',low_memory=False, parse_dates=['ticket_issued_date', 'hearing_date'])
    address = pd.read_csv('addresses.csv',encoding='ISO-8859-1',low_memory=False)
    coord   = pd.read_csv('latlons.csv',encoding='ISO-8859-1',low_memory=False)

    # now we remove columns and rows with all entries being EMPTY
    train.dropna(how='all',axis=1, inplace=True)
    train.dropna(how='all',axis=0, inplace=True)

    # Remove columns with the same values they are independent/non-correlated to/from target values
    independent = []
    for i in range(len(train.columns)):
        if len(train[train.columns[i]].unique())==1:
            independent.append(train.columns[i])

    # print('{} column was removed'.format(independent))
    train.drop(independent,axis=1,inplace=True)
    test.drop(independent,axis=1,inplace=True)

    # we see that there are a lot of columns with total unique values less than 250.Thus we can convert them into categorical data to reduce memory usage
    # to reduce memory usage we convert columns with < than 250 entries to categorical data
    for i in range(len(train.columns)):
        if len(train[train.columns[i]].unique())<250:
            train[train.columns[i]] = train[train.columns[i]].astype('category')

    # now lets see the missing number ratio in the data set
    total_null = train.isnull().sum().sort_values(ascending=False)
    per        = train.isnull().count().sort_values(ascending=False)

    # now i remove columns with missing value percentage of more than 50%
    high_mssing_data = pd.concat([total_null,total_null/per], keys=['Total_nulls','percentage_nulls'],axis=1)
    high_missing_values = high_mssing_data[high_mssing_data['percentage_nulls']>0.5].index
    train.drop(high_missing_values,axis=1,inplace=True)

    # Now we join the address to train and test data
    address = address.merge(coord,how='inner',left_on='address',right_on='address')
    train = train.merge(address,how='left',left_on='ticket_id',right_on='ticket_id')\
                    .set_index('ticket_id')
    test  = test.merge(address,how='left',left_on='ticket_id',right_on='ticket_id')\
                    .set_index('ticket_id')

    # now we reduce the features that can be replaced by the lat and lon
    latlon_replaced = ['violator_name',
           'violation_street_number', 'violation_street_name',
           'mailing_address_str_number', 'mailing_address_str_name',
           'state', 'zip_code', 'country','address','city']
    train.drop(latlon_replaced, axis=1,inplace=True)

    # Now we reduce the features even further, by suming the amount payables into one
    train['total_amt_pay'] = train[['fine_amount','admin_fee','state_fee','late_fee']].sum(axis=1).subtract(train['discount_amount'].astype(np.float64))
    test['total_amt_pay']  = test[['fine_amount','admin_fee','state_fee','late_fee']].sum(axis=1).subtract(test['discount_amount'].astype(np.float64))
    drop_payments = ['fine_amount','admin_fee','state_fee','late_fee','discount_amount']
    train.drop(drop_payments,axis=1, inplace=True)

    # drop missing values of ['lat','lon','total_amt_pay'] from the train dataset but since its not allowed in the test set,we replace it with the mean
    train.dropna(subset = ['lat','lon','total_amt_pay'],inplace=True)
    test['lat'].fillna(test.lat.mean(),inplace=True)
    test['lon'].fillna(test.lon.mean(),inplace=True)

    # Now we find the time gap between the ticket issue data and the hearing date
    train['time_delta'] = (train['hearing_date'] - train['ticket_issued_date']).dt.days
    test['time_delta']  = (test['hearing_date'] - test['ticket_issued_date']).dt.days
    drop_timedelta = ['hearing_date','ticket_issued_date'] 
    train.drop(drop_timedelta,axis=1, inplace=True)
    test.drop(drop_timedelta,axis=1, inplace=True)

    # Replace the missing values in the time delta column with the mode
    train['time_delta'].fillna(73, inplace=True)
    test['time_delta'].fillna(73,inplace=True)

    # Now remove not too important featured and make strinig features from string categories 'disposition','agancy_name'
    further_drop = ['inspector_name', 'violation_code','violation_description',
                    'payment_amount', 'balance_due','payment_status',
                    'compliance_detail']

    train.drop(further_drop,axis=1, inplace=True)
    string_features = ['disposition','agency_name'] 
    train = pd.get_dummies(train,columns = string_features,drop_first=True)
    test = pd.get_dummies(test,columns = string_features,drop_first=True)

    # taking only non-NaN values for training
    train = train[( (train['compliance']==0) | (train['compliance']==1) )]

    # trime the train data to have only the columns available in the test data
    y = train['compliance']
    X = train.drop('compliance',axis=1)

    train_feature_set = set(X)
    for feature in set(X):
        if feature not in test:
            train_feature_set.remove(feature)
    train_features = list(train_feature_set)

    X_train = X[train_features]
    test = test[train_features]

    #import necessary models to train the data
    from sklearn.preprocessing import MinMaxScaler, StandardScaler
    from sklearn.metrics import roc_auc_score
    from sklearn.ensemble import RandomForestClassifier
    import time

    from sklearn.ensemble import RandomForestRegressor
#     X_train = MinMaxScaler().fit_transform(X_train)
#     testt = MinMaxScaler().fit_transform(test)
    RF_clf = RandomForestRegressor(max_depth=6).fit(X_train,y)
    y_pred = RF_clf.predict(test)

    test['compliance'] = y_pred
    return test.compliance


In [7]:
blight_model().mean()

0.11096674526927389